In [ ]:
# !mkdir dataset
# !gdown --id 1SXrMPCv7iywBCOnCuSe7opd_p3MA7xvA --output /content/dataset/Defect_images.7z
# !gdown --id 1_7bob9HDskP0nvShsDcrjGWe1oL5z3AY --output /content/dataset/Mask_images.7z
# !gdown --id 1DIh-KtVpoLKNsaWAFdGXxORFQNxiUgQJ --output /content/dataset/Mask_images.7z

In [ ]:
!mkdir dataset
%cd dataset
!cp /content/drive/MyDrive/*.7z ./

/content/dataset


In [ ]:
# 현재 위치 확인
!pwd

/content/dataset


In [ ]:
!7z x Defect_images.7z
!7z x Mask_images.7z
!7z x NODefect_images.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 77676758 bytes (75 MiB)

Extracting archive: Defect_images.7z
--
Path = Defect_images.7z
Type = 7z
Physical Size = 77676758
Headers Size = 1620
Method = LZMA2:24
Solid = +
Blocks = 1

  0%     87% 90 - Defect_images/0093_030_01.png                                       Everything is Ok

Folders: 1
Files: 106
Size:       77674496
Compressed: 77676758

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 88311 bytes 

In [ ]:
import cv2
import os
import glob
import shutil
import random
import string
import numpy as np

In [ ]:
PATH_DEFECT = '/content/dataset/Defect_images/'
PATH_MASK = '/content/dataset/Mask_images/'
PATH_NODEFECT = '/content/dataset/NODefect_images/'

In [ ]:
# Dataset 불러오기
random.seed(0)

defect_list = glob.glob(PATH_DEFECT + '*.png')
mask_list = glob.glob(PATH_MASK + '*.png')
pass_list = glob.glob(PATH_NODEFECT + '**/*.png')

In [ ]:
'/content/dataset/Mask_images/0016_006_02_mask.png'.split('/')[-1].split('_')[0]

'0016'

In [ ]:
'/content/dataset/NODefect_images/2608691-202020u/0018_00_01.png'.split('/')[-1].split('_')[0]

'0018'

In [ ]:
# match defect - mask pairs
new_defect_list = list()
new_mask_list = list()
for defect in defect_list:
  num = defect.split('/')[-1].split('_')[0]
  for mask in mask_list:
    num_mask = mask.split('/')[-1].split('_')[0]
    if num == num_mask:
      new_defect_list.append(defect)
      new_mask_list.append(mask)
      break
defect_list = new_defect_list
mask_list = new_mask_list

# 첫 발송 데이터 생성

In [ ]:
os.path.exists('/content/dataset/1') is False

True

In [ ]:
# First dataset given
if os.path.exists('/content/dataset/1') is False:
  os.mkdir('/content/dataset/1')

for file_name in pass_list + defect_list:
  if random.randint(0,9) < 2:
    barcode = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
    shutil.copy2(file_name, '/content/dataset/1/' + barcode + '.png')

# 두번째 데이터 생성

In [ ]:
# second dataset
if os.path.exists('/content/dataset/2') is False:
  os.mkdir('/content/dataset/2')
if os.path.exists('/content/dataset/2/OK') is False:
  os.mkdir('/content/dataset/2/OK')
if os.path.exists('/content/dataset/2/FAIL') is False:
  os.mkdir('/content/dataset/2/FAIL')

idx =0
for file_name in pass_list:
  img = cv2.imread(file_name)
  height, width, _ = img.shape
  step = height // 2
  for i in range(width // step):
    w = i * step
    if w < width - height and random.randint(0, 9) < 2:
      patch = img[:, w:w+height, :]
      cv2.imwrite('/content/dataset/2/OK/%04d.png' % idx, patch)
      idx += 1

patch_list = list()
for item in zip(defect_list, mask_list):
  defect, mask = item
  img_d = cv2.imread(defect)
  img_m = cv2.imread(mask)

  height, width, _ = img_d.shape
  step = height // 2
  for i in range(width // step):
    w = i * step
    if w < width - height:
      patch = img_d[:, w:w+height, :]
      patch_m = img_m[:, w:w+height, :]
      if patch_m.sum() > 0:
        patch_list.append(patch)
random.shuffle(patch_list)
patch_list_fraction = patch_list[:len(patch_list)//3]
for idx, patch in enumerate(patch_list_fraction):
  cv2.imwrite('/content/dataset/2/FAIL/%4d.png'% idx, patch)

# 세번째 데이터 생성

In [ ]:
# third dataset
if os.path.exists('/content/dataset/3') is False:
  os.mkdir('/content/dataset/3')
if os.path.exists('/content/dataset/3/OK') is False:
  os.mkdir('/content/dataset/3/OK')
if os.path.exists('/content/dataset/3/FAIL') is False:
  os.mkdir('/content/dataset/3/FAIL')
if os.path.exists('/content/dataset/3/MASK') is False:
  os.mkdir('/content/dataset/3/MASK')

idx =0
for file_name in pass_list:
  img = cv2.imread(file_name)
  height, width, _ = img.shape
  step = height // 2
  for i in range(width // step):
    w = i * step
    if w < width - height and random.randint(0, 9) < 3:
      patch = img[:, w:w+height, :]
      cv2.imwrite('/content/dataset/3/OK/%04d.png' % idx, patch)
      idx += 1

patch_pair_list = list()
for item in zip(defect_list, mask_list):
  defect, mask = item
  img_d = cv2.imread(defect)
  img_m = cv2.imread(mask)

  height, width, _ = img_d.shape
  step = height // 2
  for i in range(width // step):
    w = i * step
    if w < width - height:
      patch_d = img_d[:, w:w+height, :]
      patch_m = img_m[:, w:w+height, :]
      if patch_d.sum() > 0:
        patch_pair_list.append((patch_d, patch_m))
random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
  patch, patch_d = pair
  cv2.imwrite('/content/dataset/3/FAIL/%4d.png'% idx, patch)
  cv2.imwrite('/content/dataset/3/MASK/%4d.png'% idx, patch_d)

# 테스트 데이터 생성

In [ ]:
# The test dataset
if os.path.exists('/content/data/') is False:
    os.mkdir('/content/data/')
if os.path.exists('/content/tfrecords/') is False:
    os.mkdir('/content/tfrecords/')
if os.path.exists('/content/model/') is False:
    os.mkdir('/content/model/')
if os.path.exists('/content/data/input_data') is False:
    os.mkdir('/content/data/input_data')
if os.path.exists('/content/data/output_csv') is False:
    os.mkdir('/content/data/output_csv')

idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2

    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 5:
            patch = img[:, w:w+height, :]
            cv2.imwrite('/content/data/input_data/ok_%04d.png' % idx, patch)
            idx += 1 

patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item

    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)

    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]

            if patch_d.sum() > 0:
                patch_pair_list.append((patch, patch_d))

random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    patch, patch_d = pair
    cv2.imwrite('/content/data/input_data/fail_%04d.png' % idx, patch)

# 실행 가능성 확인하기 (베이스라인 모델 생성)

In [ ]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

## 하이퍼파라미터 선언

In [ ]:
EPOCHS = 10
DATASET_PATH = '/content/dataset/2/'
DATASET_OK_PATH = DATASET_PATH + 'OK/*.png'
DATASET_FAIL_PATH = DATASET_PATH + 'FAIL/*.png'
RESULT_SAVE_PATH = '/content/model/'

In [ ]:
DATASET_OK_PATH, '/content/dataset/2/OK'

('/content/dataset/2/OK/*.png', '/content/dataset/2/OK')

## 단순한 모델 설정

In [ ]:
def Model():
  return Sequential([
      Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 1)),
      MaxPool2D(),
      Conv2D(64, (3,3), activation='relu'),
      MaxPool2D(),
      Conv2D(128, (3,3), activation='relu'),
      MaxPool2D(),
      Conv2D(256, (3,3), activation='relu'),
      MaxPool2D(),
      Flatten(),
      Dense(1, activation='sigmoid')
  ])

## 데이터셋 불러오기

In [ ]:
def preprocess(file_name):
  img = tf.io.read_file(file_name)
  img = tf.image.decode_png(img, channels=1)
  return tf.image.convert_image_dtype(img, tf.float32)

In [ ]:
ok_list = glob.glob(DATASET_OK_PATH)
ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list))
ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

In [ ]:
fail_list = glob.glob(DATASET_FAIL_PATH)
ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list))
ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

In [ ]:
ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

In [ ]:
# train valid 데이터 나누기
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_valid = ds.skip(train_size).batch(32)

In [ ]:
# 모델 생성 및 학습
model = Model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 128)      0

In [ ]:
model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

Epoch 1/10
42/42 [==============================] - 138s 3s/step - loss: 0.6329 - accuracy: 0.6495 - val_loss: 0.7256 - val_accuracy: 0.5546
Epoch 2/10
42/42 [==============================] - 130s 3s/step - loss: 0.6541 - accuracy: 0.6246 - val_loss: 0.5675 - val_accuracy: 0.3891
Epoch 3/10
42/42 [==============================] - 130s 3s/step - loss: 0.4883 - accuracy: 0.7772 - val_loss: 0.3265 - val_accuracy: 0.8944
Epoch 4/10
42/42 [==============================] - 131s 3s/step - loss: 0.6743 - accuracy: 0.5687 - val_loss: 0.6757 - val_accuracy: 0.5739
Epoch 5/10
42/42 [==============================] - 134s 3s/step - loss: 0.6210 - accuracy: 0.6579 - val_loss: 0.8063 - val_accuracy: 0.6813
Epoch 6/10
42/42 [==============================] - 135s 3s/step - loss: 0.3294 - accuracy: 0.8731 - val_loss: 0.1136 - val_accuracy: 0.9912
Epoch 7/10
42/42 [==============================] - 134s 3s/step - loss: 0.0495 - accuracy: 0.9872 - val_loss: 0.0249 - val_accuracy: 0.9912
Epoch 8/10
42

# 결과를 이미지로 저장

In [ ]:
def mkdir(path):
  if os.path.exists(path) is False:
    os.mkdir(path)

In [ ]:
mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP')
mkdir(RESULT_SAVE_PATH + '/TN')
mkdir(RESULT_SAVE_PATH + '/FP')
mkdir(RESULT_SAVE_PATH + '/FN')

In [31]:
index = 0
for imags, labels in ds_valid:
  preds = model.predict(imags)
  for idx in range(imags.shape[0]):
    gt = labels[idx].numpy()
    y = preds[idx]
    if gt == 1 and y > 0.5:
      path = RESULT_SAVE_PATH + '/TP'
    elif gt == 1 and y <= 0.5:
      path = RESULT_SAVE_PATH + '/FN'
    elif gt == 0 and y > 0.5:
      path = RESULT_SAVE_PATH + '/FP'
    else:
      path = RESULT_SAVE_PATH + '/TN'
    cv2.imwrite(path + '/%.f4_%04d.png' % (y, index), imags[idx].numpy() * 255)
    index += 1
    

# TFRecord Builder

In [32]:
import glob
import os
import tensorflow as tf
import cv2

In [44]:
# Paths and Hyperparameters
DATASET_OK_PATH = '/content/dataset/3/OK/*.png'
DATASET_FAIL_PATH = '/content/dataset/3/FAIL/*.png'
TRFECORD_PATH = '/content/tfrecords/'
IMAGE_PER_TFRCEORD = 100

In [47]:
ok_list = glob.glob(DATASET_OK_PATH)
fail_list = glob.glob(DATASET_FAIL_PATH)

num_ok = len(ok_list)
num_fail = len(fail_list)

In [50]:
num_ok, num_fail, num_ok // num_fail

(1284, 3148, 0)

In [37]:
# Oversampling
fail_list_new = list()
for _ in range(num_ok // num_fail):
  fail_list_new += fail_list

fail_list_new += fail_list[: num_ok % num_fail]
fail_list = fail_list_new

ok_label = [0] * len(ok_list)
fail_label = [1] * len(fail_list)

file_list = ok_list + fail_list
label_list = ok_label + fail_label

## TFRecord funtions

In [40]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def image_example(image_string, label):
    image_shape = tf.image.decode_image(image_string).shape

    feature = {
        'height': _int64_feature(image_shape[0]),
        'width': _int64_feature(image_shape[1]),
        'depth': _int64_feature(image_shape[2]),
        'label': _int64_feature(label),
        'image_raw': _bytes_feature(image_string),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

In [51]:
from tensorflow.python.lib.io.tf_record import TFRecordCompressionType
if os.path.exists(TRFECORD_PATH) is False:
  os.mkdir(TRFECORD_PATH)

num_tfrecords = len(file_list) // IMAGE_PER_TFRCEORD
if len(file_list) % IMAGE_PER_TFRCEORD != 0:
  num_tfrecords += 1

for idx in range(num_tfrecords):
  idx0 = idx * IMAGE_PER_TFRCEORD
  idx1 = idx0 + IMAGE_PER_TFRCEORD
  record_file = TRFECORD_PATH + '%05d.tfrecords' % idx
  with tf.io.TFRecordWriter(record_file) as writer:
    for filename, label in zip(file_list[idx0:idx1], 
                               label_list[idx0:idx1]):
      image_string = open(filename, 'rb').read()
      tf_example = image_example(image_string, label)
      writer.write(tf_example.SerializeToString())


# 모델 정의

In [ ]:
!pip install tensorflow_addons

In [55]:
import glob
import os
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Concatenate, Flatten, Dense

In [56]:
EPOCHS = 1000
RESULT_SAVE_PATH = '/content/results/'

# Inception-based 모델정의

In [57]:
def Model():
  def inception(filters):
    def subnetwork(x):
      h1 = Conv2D(filters, (1,1), padding='same', activation='relu')(x)
      h1 = MaxPool2D()(h1)

      h2 = Conv2D(filters // 2, (1,1), padding='same', activation='relu')(x)
      h2 = Conv2D(filters, (3,3), padding='same', activation='relu')(h2)
      h2 = MaxPool2D()(h2)

      h3 = Conv2D(filters // 2, (1,1), padding='same', activation='relu')(x)
      h3 = Conv2D(filters, (5,5), padding='same', activation='relu')(h3)
      h3 = MaxPool2D()(h3)
      return Concatenate()([h1, h2, h3])
    return subnetwork

  x = tf.keras.Input(shape=(256, 256, 3))
  h = inception(16)(x)
  h = inception(32)(h)
  h = inception(32)(h)
  h = inception(32)(h)
  h = inception(32)(h)
  h = Flatten()(h)
  h = Dense(1024, activation='relu')(h)
  y = Dense(1, activation='sigmoid')(h)
  return tf.keras.Model(inputs=x, outputs=y)


# Data Augmenation

In [58]:
def preprocess(img):
  return tf.image.convert_image_dtype(img, tf.float32)

In [59]:
def augmentation(img, label):
  def flip(x):
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)
    return x

  def rotate(x):
    x = tf.cond(tf.random_uniform(shpae=[], minval=0.0, maxval=1.0, dtype=tf.float32) > 0.5,
                lambda: tfa.image.rotate(x, 
                  tf.random_uniform(shpae=[], minval=0.0, maxval=1.0, dtype=tf.float32),
                  interpolation='BILINEAR'),
                lambda: x)
    return x

  def translation(x):
    dx = tf.random.uniform(shape=[], minval=-10.0, maxval=10.0, dtype=tf.float32)
    dy = tf.random.uniform(shape=[], minval=-10.0, maxval=10.0, dtype=tf.float32)
    x = tf.cond(tf.random_uniform(shpae=[], minval=0.0, maxval=1.0, dtype=tf.float32) > 0.5,
                lambda: tfa.image_transform(x,
                                            [0,0,dx,0,0,dy,0,0],
                                            interpolation='BILINEAR'),
                lambda: x)
    return x
  
  img = flip(img)
  img = rotate(img)
  img = translation(img)
  return img, label


In [60]:
tffiles = glob.glob('/content/tfrecords/*')
raw_image_dataset = tf.data.TFRecordDataset(tffiles)

image_feature_description = {
  'height': tf.io.FixedLenFeature([], tf.int64),
  'width': tf.io.FixedLenFeature([], tf.int64),
  'depth': tf.io.FixedLenFeature([], tf.int64),
  'label': tf.io.FixedLenFeature([], tf.int64),
  'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  return tf.io.parse_single_example(example_proto, image_feature_description)

def _parse_image_label(parsed_dataset):
  return preprocess(tf.image.decode_png(parsed_dataset['image_raw'])), parsed_dataset['label']

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
dataset = parsed_image_dataset.map(_parse_image_label)

In [61]:
model = Model()
model.compile(optimizer='adam', loss='banary_crossentropy', metrics=['accuracy'])

In [62]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_5 (Conv2D)              (None, 256, 256, 8)  32          ['input_1[0][0]']                
                                                                                                  
 conv2d_7 (Conv2D)              (None, 256, 256, 8)  32          ['input_1[0][0]']                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 256, 256, 16  64          ['input_1[0][0]']            

In [63]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=1)

In [64]:
history = model.fit(ds_train, validation_data=ds_valid,
                    epochs=EPOCHS,
                    callbacks=[earlystopping])

Epoch 1/1000


ValueError: ignored